In [1]:
pip install geopy

     |████████████████████████████████| 112kB 22.7MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [2]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


## data for neighborhoods at Manhattan

In [3]:
!wget -q -O 'newyork_data.json' https://cocl.us/new_york_dataset
print('Data downloaded!')

Data downloaded!


In [4]:
with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)

In [5]:
neighborhoods_data = newyork_data['features']

In [6]:
# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)

In [7]:
neighborhoods.head()

,Borough,Neighborhood,Latitude,Longitude


In [8]:
for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)

In [9]:
neighborhoods.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


In [10]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(neighborhoods['Borough'].unique()),
        neighborhoods.shape[0]
    )
)

The dataframe has 5 boroughs and 306 neighborhoods.


In [11]:
address = 'New York City, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of New York City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of New York City are 40.7127281, -74.0060152.


In [12]:
manhattan_data = neighborhoods[neighborhoods['Borough'] == 'Manhattan'].reset_index(drop=True)
manhattan_data.head()

,Borough,Neighborhood,Latitude,Longitude
0,Manhattan,Marble Hill,40.876551,-73.910660
1,Manhattan,Chinatown,40.715618,-73.994279
2,Manhattan,Washington Heights,40.851903,-73.936900
3,Manhattan,Inwood,40.867684,-73.921210
4,Manhattan,Hamilton Heights,40.823604,-73.949688


In [50]:
manhattan_data.shape

(40, 4)

In [24]:
address = 'Manhattan, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Manhattan are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Manhattan are 40.7896239, -73.9598939.


In [52]:
# create map of Manhattan using latitude and longitude values
map_manhattan = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(manhattan_data['Latitude'], manhattan_data['Longitude'], manhattan_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_manhattan)  
    
map_manhattan

In [25]:
CLIENT_ID = 'U3L2CZO4I2CN3U4KQ3UKQBTZQYTI5FGONO3IEPZZYXU0IFIE' # your Foursquare ID
CLIENT_SECRET = 'TI42ULVVCXHPAFA4XQLBT5FTJ24PVPPDVQUTLIDBO2L2NRZS' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: U3L2CZO4I2CN3U4KQ3UKQBTZQYTI5FGONO3IEPZZYXU0IFIE
CLIENT_SECRET:TI42ULVVCXHPAFA4XQLBT5FTJ24PVPPDVQUTLIDBO2L2NRZS


In [26]:
# type your answer here
LIMIT = 20
radius = 500
categoryId = '4d4b7105d754a06374d81259'



In [27]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [28]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&categoryId={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET,
            categoryId,
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name'],
            v['venue']['id']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category',
                        'Venue Id']
    
    return(nearby_venues)

In [29]:
# type your answer here

manhattan_venues = getNearbyVenues(names=manhattan_data['Neighborhood'],
                                   latitudes=manhattan_data['Latitude'],
                                   longitudes=manhattan_data['Longitude']
                                  )



Marble Hill
Chinatown
Washington Heights
Inwood
Hamilton Heights
Manhattanville
Central Harlem
East Harlem
Upper East Side
Yorkville
Lenox Hill
Roosevelt Island
Upper West Side
Lincoln Square
Clinton
Midtown
Murray Hill
Chelsea
Greenwich Village
East Village
Lower East Side
Tribeca
Little Italy
Soho
West Village
Manhattan Valley
Morningside Heights
Gramercy
Battery Park City
Financial District
Carnegie Hill
Noho
Civic Center
Midtown South
Sutton Place
Turtle Bay
Tudor City
Stuyvesant Town
Flatiron
Hudson Yards


In [214]:
print(manhattan_venues.shape)
manhattan_venues.head()

(771, 8)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,Venue Id
0,Marble Hill,40.876551,-73.91066,Arturo's,40.874412,-73.910271,Pizza Place,4b4429abf964a52037f225e3
1,Marble Hill,40.876551,-73.91066,Tibbett Diner,40.880404,-73.908937,Diner,4b79cc46f964a520c5122fe3
2,Marble Hill,40.876551,-73.91066,Dunkin',40.877136,-73.906666,Donut Shop,4b5357adf964a520319827e3
3,Marble Hill,40.876551,-73.91066,Land & Sea Restaurant,40.877885,-73.905873,Seafood Restaurant,4b9c9c6af964a520b27236e3
4,Marble Hill,40.876551,-73.91066,Subway Sandwiches,40.874667,-73.909586,Sandwich Place,4b4f7b65f964a5205a0827e3


In [215]:
manhattan_venues.to_csv('dining_venue_list.csv')

In [31]:
manhattan_venues.shape

(771, 8)

In [33]:
lor = manhattan_venues['Venue Id'].tolist()

In [35]:
len(lor[458:])

313

In [48]:
loprice=[]
lolikes=[]


# data for restaurants detail, price and likes count

In [49]:
for r_id in lor[458:460]:
    url_loop= 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(
    r_id,
    CLIENT_ID, 
    CLIENT_SECRET,
    VERSION, 
    )
    r_detail = requests.get(url_loop).json()
    try:
        loprice.append(r_detail['response']['venue']['price']['tier'])
    except Exception as e:
        loprice.append(-1)
    try:
        lolikes.append(r_detail['response']['venue']['likes']['count'])
    except  Exception as e:
        lolikes.append(-1)
    print(url_loop)
    
    

https://api.foursquare.com/v2/venues/5a7cb7bc2105c32f7cdf9fef?client_id=U3L2CZO4I2CN3U4KQ3UKQBTZQYTI5FGONO3IEPZZYXU0IFIE&client_secret=TI42ULVVCXHPAFA4XQLBT5FTJ24PVPPDVQUTLIDBO2L2NRZS&v=20180605
https://api.foursquare.com/v2/venues/5431872b498eec43384fd39d?client_id=U3L2CZO4I2CN3U4KQ3UKQBTZQYTI5FGONO3IEPZZYXU0IFIE&client_secret=TI42ULVVCXHPAFA4XQLBT5FTJ24PVPPDVQUTLIDBO2L2NRZS&v=20180605


In [36]:
pl1 = pd.read_csv('price_likes.csv',index_col=0)

In [37]:
pl1.head()

,price_tier,likes_count
0,1,16
1,1,47
2,1,67
3,2,42
4,1,1


In [53]:
pl1

,price_tier,likes_count
0,1,16
1,1,47
2,1,67
3,2,42
4,1,1
5,2,4
6,1,3
7,4,8
8,1,0
9,1,0


In [43]:
soprice = pd.Series(data=loprice, name='price_tier')

In [44]:
solikes = pd.Series(data=lolikes, name='likes_count')

In [45]:
price_likes = pd.DataFrame({'price_tier':soprice,'likes_count':solikes})

In [47]:
price_likes

,price_tier,likes_count
0,-1,-1


In [59]:
t1 = manhattan_venues.join(pl1)

ValueError: columns overlap but no suffix specified: Index(['price_tier', 'likes_count'], dtype='object')

In [55]:
manhattan_venues=t1[(t1['price_tier']!=-1)&(t1['likes_count']!=-1)]

In [58]:
manhattan_venues[['Neighborhood','price_tier','likes_count']]

,Neighborhood,price_tier,likes_count
0,Marble Hill,1,16
1,Marble Hill,1,47
2,Marble Hill,1,67
3,Marble Hill,2,42
4,Marble Hill,1,1
5,Marble Hill,2,4
6,Marble Hill,1,3
7,Marble Hill,4,8
8,Marble Hill,1,0
9,Marble Hill,1,0


In [56]:
manhattan_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,Venue Id,price_tier,likes_count
0,Marble Hill,40.876551,-73.91066,Arturo's,40.874412,-73.910271,Pizza Place,4b4429abf964a52037f225e3,1,16
1,Marble Hill,40.876551,-73.91066,Tibbett Diner,40.880404,-73.908937,Diner,4b79cc46f964a520c5122fe3,1,47
2,Marble Hill,40.876551,-73.91066,Dunkin',40.877136,-73.906666,Donut Shop,4b5357adf964a520319827e3,1,67
3,Marble Hill,40.876551,-73.91066,Land & Sea Restaurant,40.877885,-73.905873,Seafood Restaurant,4b9c9c6af964a520b27236e3,2,42
4,Marble Hill,40.876551,-73.91066,Subway Sandwiches,40.874667,-73.909586,Sandwich Place,4b4f7b65f964a5205a0827e3,1,1


In [60]:
manhattan_venues.shape

(432, 10)

In [61]:
from sklearn.preprocessing import StandardScaler

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


array([[-1.15763226e+00, -5.86886315e-01],
       [-1.15763226e+00, -5.04396139e-01],
       [-1.15763226e+00, -4.51176670e-01],
       [ 1.35554129e-02, -5.17701006e-01],
       [-1.15763226e+00, -6.26800917e-01],
       [ 1.35554129e-02, -6.18817997e-01],
       [-1.15763226e+00, -6.21478970e-01],
       [ 2.35593076e+00, -6.08174103e-01],
       [-1.15763226e+00, -6.29461890e-01],
       [-1.15763226e+00, -6.29461890e-01],
       [-1.15763226e+00, -6.29461890e-01],
       [ 1.35554129e-02, -6.18817997e-01],
       [-1.15763226e+00, -6.26800917e-01],
       [-1.15763226e+00, -6.26800917e-01],
       [ 1.35554129e-02,  8.15446687e-01],
       [ 1.35554129e-02,  1.56584120e+00],
       [-1.15763226e+00, -2.00989730e-02],
       [-1.15763226e+00,  7.01024829e-01],
       [ 1.35554129e-02,  1.27845607e+00],
       [-1.15763226e+00, -5.68259501e-01],
       [-1.15763226e+00, -5.46971714e-01],
       [-1.15763226e+00, -1.18554990e-01],
       [-1.15763226e+00, -3.52720653e-01],
       [-1.

In [228]:
print('There are {} uniques categories.'.format(len(manhattan_venues['Venue Category'].unique())))

There are 65 uniques categories.


# process category data into dummies

In [62]:
# one hot encoding
manhattan_onehot = pd.get_dummies(manhattan_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
manhattan_onehot['Neighborhood'] = manhattan_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [manhattan_onehot.columns[-1]] + list(manhattan_onehot.columns[:-1])
manhattan_onehot = manhattan_onehot[fixed_columns]

manhattan_onehot.head()

,Neighborhood,Afghan Restaurant,African Restaurant,American Restaurant,Argentinian Restaurant,Asian Restaurant,BBQ Joint,Bagel Shop,Bakery,Breakfast Spot,Burger Joint,Burrito Place,Cafeteria,Café,Caribbean Restaurant,Chinese Restaurant,Cuban Restaurant,Deli / Bodega,Diner,Donut Shop,Dumpling Restaurant,English Restaurant,Ethiopian Restaurant,Falafel Restaurant,Fast Food Restaurant,Filipino Restaurant,Food Truck,French Restaurant,Fried Chicken Joint,Gastropub,German Restaurant,Greek Restaurant,Hawaiian Restaurant,Hot Dog Joint,Hotpot Restaurant,Indian Restaurant,Israeli Restaurant,Italian Restaurant,Japanese Restaurant,Jewish Restaurant,Korean Restaurant,Latin American Restaurant,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Moroccan Restaurant,New American Restaurant,Noodle House,Peruvian Restaurant,Pizza Place,Ramen Restaurant,Restaurant,Salad Place,Sandwich Place,Scandinavian Restaurant,Seafood Restaurant,Snack Place,Southern / Soul Food Restaurant,Spanish Restaurant,Steakhouse,Sushi Restaurant,Swiss Restaurant,Taco Place,Tapas Restaurant,Thai Restaurant,Turkish Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant
0,Marble Hill,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Marble Hill,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Marble Hill,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Marble Hill,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
4,Marble Hill,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [63]:
manhattan_onehot.shape

(432, 68)

In [64]:
manhattan_grouped = manhattan_onehot.groupby('Neighborhood').mean().reset_index()
manhattan_grouped.head()

,Neighborhood,Afghan Restaurant,African Restaurant,American Restaurant,Argentinian Restaurant,Asian Restaurant,BBQ Joint,Bagel Shop,Bakery,Breakfast Spot,Burger Joint,Burrito Place,Cafeteria,Café,Caribbean Restaurant,Chinese Restaurant,Cuban Restaurant,Deli / Bodega,Diner,Donut Shop,Dumpling Restaurant,English Restaurant,Ethiopian Restaurant,Falafel Restaurant,Fast Food Restaurant,Filipino Restaurant,Food Truck,French Restaurant,Fried Chicken Joint,Gastropub,German Restaurant,Greek Restaurant,Hawaiian Restaurant,Hot Dog Joint,Hotpot Restaurant,Indian Restaurant,Israeli Restaurant,Italian Restaurant,Japanese Restaurant,Jewish Restaurant,Korean Restaurant,Latin American Restaurant,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Moroccan Restaurant,New American Restaurant,Noodle House,Peruvian Restaurant,Pizza Place,Ramen Restaurant,Restaurant,Salad Place,Sandwich Place,Scandinavian Restaurant,Seafood Restaurant,Snack Place,Southern / Soul Food Restaurant,Spanish Restaurant,Steakhouse,Sushi Restaurant,Swiss Restaurant,Taco Place,Tapas Restaurant,Thai Restaurant,Turkish Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant
0,Central Harlem,0.0,0.15,0.100000,0.0,0.000000,0.0,0.05,0.000000,0.0,0.0,0.0,0.05,0.05,0.05,0.100000,0.00,0.0,0.0,0.00,0.0,0.000000,0.05,0.0,0.0,0.0,0.0,0.100000,0.1,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.00,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.050000,0.0,0.000000,0.0,0.000000,0.0,0.100000,0.0,0.05,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0
1,Chelsea,0.0,0.00,0.166667,0.0,0.055556,0.0,0.00,0.000000,0.0,0.0,0.0,0.00,0.00,0.00,0.055556,0.00,0.0,0.0,0.00,0.0,0.000000,0.00,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.055556,0.055556,0.111111,0.111111,0.0,0.0,0.00,0.000000,0.055556,0.055556,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.055556,0.0,0.111111,0.0,0.00,0.000000,0.000000,0.0,0.0,0.055556,0.055556,0.000000,0.0,0.0,0.0
2,Chinatown,0.0,0.00,0.058824,0.0,0.058824,0.0,0.00,0.058824,0.0,0.0,0.0,0.00,0.00,0.00,0.176471,0.00,0.0,0.0,0.00,0.0,0.058824,0.00,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.058824,0.0,0.0,0.0,0.000000,0.000000,0.058824,0.000000,0.0,0.0,0.00,0.000000,0.000000,0.000000,0.0,0.058824,0.117647,0.000000,0.058824,0.0,0.000000,0.0,0.117647,0.0,0.000000,0.0,0.00,0.058824,0.000000,0.0,0.0,0.000000,0.000000,0.058824,0.0,0.0,0.0
3,Clinton,0.0,0.00,0.266667,0.0,0.000000,0.0,0.00,0.000000,0.0,0.0,0.0,0.00,0.00,0.00,0.000000,0.00,0.0,0.0,0.00,0.0,0.000000,0.00,0.0,0.0,0.0,0.0,0.066667,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.066667,0.000000,0.0,0.0,0.00,0.066667,0.000000,0.000000,0.0,0.133333,0.000000,0.066667,0.066667,0.0,0.066667,0.0,0.066667,0.0,0.066667,0.0,0.00,0.000000,0.066667,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0
4,East Harlem,0.0,0.00,0.000000,0.0,0.000000,0.0,0.00,0.100000,0.0,0.0,0.0,0.00,0.05,0.00,0.000000,0.05,0.0,0.0,0.05,0.0,0.000000,0.00,0.0,0.0,0.0,0.0,0.050000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.15,0.000000,0.200000,0.000000,0.0,0.000000,0.000000,0.000000,0.050000,0.0,0.000000,0.0,0.050000,0.0,0.000000,0.0,0.00,0.050000,0.050000,0.0,0.0,0.050000,0.000000,0.100000,0.0,0.0,0.0


In [65]:
manhattan_grouped.shape

(24, 68)

# normalize price and likes count 

In [66]:
m11 = manhattan_venues[['Neighborhood','price_tier','likes_count']]

In [67]:
m12 = m11.groupby('Neighborhood').mean()

In [68]:
m12.head()

,price_tier,likes_count
Neighborhood,,
Central Harlem,2.000000,37.850000
Chelsea,2.277778,513.611111
Chinatown,1.529412,265.823529
Clinton,2.400000,204.133333
East Harlem,1.600000,41.450000


In [69]:
X = m12.values[:,:]
cluster_dataset = StandardScaler().fit_transform(X)
cluster_dataset[:10]

array([[ 0.06427182, -0.98375749],
       [ 0.82841609,  1.33200242],
       [-1.23027847,  0.12590017],
       [ 1.16463957, -0.17437591],
       [-1.03609593, -0.96623454],
       [-0.24138589,  1.69652294],
       [ 0.49862751,  1.15661642],
       [-1.44873383, -0.97426589],
       [-1.09401002, -1.04169336],
       [ 0.20905705, -0.25521032]])

In [70]:
m13 = pd.DataFrame(cluster_dataset,columns=['price_tier','likes_count'])

In [71]:
m13.head()

,price_tier,likes_count
0,0.064272,-0.983757
1,0.828416,1.332002
2,-1.230278,0.125900
3,1.164640,-0.174376
4,-1.036096,-0.966235


In [72]:
insert1 = m13.iloc[:,0]

In [73]:
insert2 = m13.iloc[:,1]

In [74]:
manhattan_grouped.insert(1,'price_tier',insert1)

In [75]:
manhattan_grouped.insert(2,'likes_count',insert2)

# manhattan_grouped is the data that ready to be process by Kmeans

In [76]:
manhattan_grouped.head()

,Neighborhood,price_tier,likes_count,Afghan Restaurant,African Restaurant,American Restaurant,Argentinian Restaurant,Asian Restaurant,BBQ Joint,Bagel Shop,Bakery,Breakfast Spot,Burger Joint,Burrito Place,Cafeteria,Café,Caribbean Restaurant,Chinese Restaurant,Cuban Restaurant,Deli / Bodega,Diner,Donut Shop,Dumpling Restaurant,English Restaurant,Ethiopian Restaurant,Falafel Restaurant,Fast Food Restaurant,Filipino Restaurant,Food Truck,French Restaurant,Fried Chicken Joint,Gastropub,German Restaurant,Greek Restaurant,Hawaiian Restaurant,Hot Dog Joint,Hotpot Restaurant,Indian Restaurant,Israeli Restaurant,Italian Restaurant,Japanese Restaurant,Jewish Restaurant,Korean Restaurant,Latin American Restaurant,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Moroccan Restaurant,New American Restaurant,Noodle House,Peruvian Restaurant,Pizza Place,Ramen Restaurant,Restaurant,Salad Place,Sandwich Place,Scandinavian Restaurant,Seafood Restaurant,Snack Place,Southern / Soul Food Restaurant,Spanish Restaurant,Steakhouse,Sushi Restaurant,Swiss Restaurant,Taco Place,Tapas Restaurant,Thai Restaurant,Turkish Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant
0,Central Harlem,0.064272,-0.983757,0.0,0.15,0.100000,0.0,0.000000,0.0,0.05,0.000000,0.0,0.0,0.0,0.05,0.05,0.05,0.100000,0.00,0.0,0.0,0.00,0.0,0.000000,0.05,0.0,0.0,0.0,0.0,0.100000,0.1,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.00,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.050000,0.0,0.000000,0.0,0.000000,0.0,0.100000,0.0,0.05,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0
1,Chelsea,0.828416,1.332002,0.0,0.00,0.166667,0.0,0.055556,0.0,0.00,0.000000,0.0,0.0,0.0,0.00,0.00,0.00,0.055556,0.00,0.0,0.0,0.00,0.0,0.000000,0.00,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.055556,0.055556,0.111111,0.111111,0.0,0.0,0.00,0.000000,0.055556,0.055556,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.055556,0.0,0.111111,0.0,0.00,0.000000,0.000000,0.0,0.0,0.055556,0.055556,0.000000,0.0,0.0,0.0
2,Chinatown,-1.230278,0.125900,0.0,0.00,0.058824,0.0,0.058824,0.0,0.00,0.058824,0.0,0.0,0.0,0.00,0.00,0.00,0.176471,0.00,0.0,0.0,0.00,0.0,0.058824,0.00,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.058824,0.0,0.0,0.0,0.000000,0.000000,0.058824,0.000000,0.0,0.0,0.00,0.000000,0.000000,0.000000,0.0,0.058824,0.117647,0.000000,0.058824,0.0,0.000000,0.0,0.117647,0.0,0.000000,0.0,0.00,0.058824,0.000000,0.0,0.0,0.000000,0.000000,0.058824,0.0,0.0,0.0
3,Clinton,1.164640,-0.174376,0.0,0.00,0.266667,0.0,0.000000,0.0,0.00,0.000000,0.0,0.0,0.0,0.00,0.00,0.00,0.000000,0.00,0.0,0.0,0.00,0.0,0.000000,0.00,0.0,0.0,0.0,0.0,0.066667,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.066667,0.000000,0.0,0.0,0.00,0.066667,0.000000,0.000000,0.0,0.133333,0.000000,0.066667,0.066667,0.0,0.066667,0.0,0.066667,0.0,0.066667,0.0,0.00,0.000000,0.066667,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0
4,East Harlem,-1.036096,-0.966235,0.0,0.00,0.000000,0.0,0.000000,0.0,0.00,0.100000,0.0,0.0,0.0,0.00,0.05,0.00,0.000000,0.05,0.0,0.0,0.05,0.0,0.000000,0.00,0.0,0.0,0.0,0.0,0.050000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.15,0.000000,0.200000,0.000000,0.0,0.000000,0.000000,0.000000,0.050000,0.0,0.000000,0.0,0.050000,0.0,0.000000,0.0,0.00,0.050000,0.050000,0.0,0.0,0.050000,0.000000,0.100000,0.0,0.0,0.0


In [77]:
num_top_venues = 5

for hood in manhattan_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = manhattan_grouped[manhattan_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[3:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Central Harlem----
                 venue  freq
0   African Restaurant  0.15
1   Seafood Restaurant  0.10
2  American Restaurant  0.10
3    French Restaurant  0.10
4  Fried Chicken Joint  0.10


----Chelsea----
                 venue  freq
0  American Restaurant  0.17
1  Japanese Restaurant  0.11
2   Italian Restaurant  0.11
3   Seafood Restaurant  0.11
4       Sandwich Place  0.06


----Chinatown----
                venue  freq
0  Chinese Restaurant  0.18
1      Sandwich Place  0.12
2        Noodle House  0.12
3  Italian Restaurant  0.06
4  Spanish Restaurant  0.06


----Clinton----
                     venue  freq
0      American Restaurant  0.27
1  New American Restaurant  0.13
2       Italian Restaurant  0.07
3              Pizza Place  0.07
4      Peruvian Restaurant  0.07


----East Harlem----
                       venue  freq
0         Mexican Restaurant  0.20
1  Latin American Restaurant  0.15
2            Thai Restaurant  0.10
3                     Bakery  0.10
4         

In [78]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[3:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [79]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Restaurant'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Restaurant'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = manhattan_grouped['Neighborhood']

for ind in np.arange(manhattan_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(manhattan_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Restaurant,2nd Most Common Restaurant,3rd Most Common Restaurant,4th Most Common Restaurant,5th Most Common Restaurant,6th Most Common Restaurant,7th Most Common Restaurant,8th Most Common Restaurant,9th Most Common Restaurant,10th Most Common Restaurant
0,Central Harlem,African Restaurant,Chinese Restaurant,American Restaurant,Fried Chicken Joint,French Restaurant,Seafood Restaurant,Café,Ethiopian Restaurant,Bagel Shop,Pizza Place
1,Chelsea,American Restaurant,Japanese Restaurant,Italian Restaurant,Seafood Restaurant,Chinese Restaurant,Indian Restaurant,Mexican Restaurant,Sandwich Place,Israeli Restaurant,Asian Restaurant
2,Chinatown,Chinese Restaurant,Noodle House,Sandwich Place,Pizza Place,Spanish Restaurant,Italian Restaurant,English Restaurant,New American Restaurant,Bakery,Greek Restaurant
3,Clinton,American Restaurant,New American Restaurant,Italian Restaurant,Mediterranean Restaurant,Pizza Place,Restaurant,Sandwich Place,Seafood Restaurant,Peruvian Restaurant,Steakhouse
4,East Harlem,Mexican Restaurant,Latin American Restaurant,Thai Restaurant,Bakery,Pizza Place,French Restaurant,Taco Place,Café,Steakhouse,Spanish Restaurant


In [80]:
m12.head()

,price_tier,likes_count
Neighborhood,,
Central Harlem,2.000000,37.850000
Chelsea,2.277778,513.611111
Chinatown,1.529412,265.823529
Clinton,2.400000,204.133333
East Harlem,1.600000,41.450000


In [82]:
neighborhoods_venues_sorted.insert(1,'price_tier',m12['price_tier'].tolist())

In [83]:
neighborhoods_venues_sorted.insert(2,'likes_count',m12['likes_count'].tolist())

In [84]:
neighborhoods_venues_sorted.head()

,Neighborhood,price_tier,likes_count,1st Most Common Restaurant,2nd Most Common Restaurant,3rd Most Common Restaurant,4th Most Common Restaurant,5th Most Common Restaurant,6th Most Common Restaurant,7th Most Common Restaurant,8th Most Common Restaurant,9th Most Common Restaurant,10th Most Common Restaurant
0,Central Harlem,2.000000,37.850000,African Restaurant,Chinese Restaurant,American Restaurant,Fried Chicken Joint,French Restaurant,Seafood Restaurant,Café,Ethiopian Restaurant,Bagel Shop,Pizza Place
1,Chelsea,2.277778,513.611111,American Restaurant,Japanese Restaurant,Italian Restaurant,Seafood Restaurant,Chinese Restaurant,Indian Restaurant,Mexican Restaurant,Sandwich Place,Israeli Restaurant,Asian Restaurant
2,Chinatown,1.529412,265.823529,Chinese Restaurant,Noodle House,Sandwich Place,Pizza Place,Spanish Restaurant,Italian Restaurant,English Restaurant,New American Restaurant,Bakery,Greek Restaurant
3,Clinton,2.400000,204.133333,American Restaurant,New American Restaurant,Italian Restaurant,Mediterranean Restaurant,Pizza Place,Restaurant,Sandwich Place,Seafood Restaurant,Peruvian Restaurant,Steakhouse
4,East Harlem,1.600000,41.450000,Mexican Restaurant,Latin American Restaurant,Thai Restaurant,Bakery,Pizza Place,French Restaurant,Taco Place,Café,Steakhouse,Spanish Restaurant


In [269]:
manhattan_grouped.head()

,Neighborhood,price_tier,likes_count,Afghan Restaurant,African Restaurant,American Restaurant,Argentinian Restaurant,Asian Restaurant,BBQ Joint,Bagel Shop,Bakery,Breakfast Spot,Burger Joint,Burrito Place,Cafeteria,Café,Caribbean Restaurant,Chinese Restaurant,Cuban Restaurant,Deli / Bodega,Diner,Donut Shop,Dumpling Restaurant,English Restaurant,Ethiopian Restaurant,Falafel Restaurant,Filipino Restaurant,Food Truck,French Restaurant,Fried Chicken Joint,Gastropub,German Restaurant,Greek Restaurant,Hawaiian Restaurant,Hot Dog Joint,Hotpot Restaurant,Indian Restaurant,Italian Restaurant,Japanese Restaurant,Jewish Restaurant,Korean Restaurant,Latin American Restaurant,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Moroccan Restaurant,New American Restaurant,Noodle House,Peruvian Restaurant,Pizza Place,Ramen Restaurant,Restaurant,Salad Place,Sandwich Place,Scandinavian Restaurant,Seafood Restaurant,Snack Place,Southern / Soul Food Restaurant,Spanish Restaurant,Steakhouse,Sushi Restaurant,Swiss Restaurant,Taco Place,Tapas Restaurant,Thai Restaurant,Turkish Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant
0,Central Harlem,0.064272,-0.983757,0.0,0.15,0.100000,0.0,0.000000,0.0,0.05,0.000000,0.0,0.0,0.0,0.05,0.05,0.05,0.100000,0.00,0.0,0.0,0.00,0.0,0.000000,0.05,0.0,0.0,0.0,0.100000,0.1,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.00,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.050000,0.0,0.000000,0.0,0.000000,0.0,0.100000,0.0,0.05,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0
1,Chelsea,0.828416,1.332002,0.0,0.00,0.055556,0.0,0.055556,0.0,0.00,0.055556,0.0,0.0,0.0,0.00,0.00,0.00,0.055556,0.00,0.0,0.0,0.00,0.0,0.000000,0.00,0.0,0.0,0.0,0.055556,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.055556,0.111111,0.111111,0.0,0.0,0.00,0.000000,0.055556,0.055556,0.0,0.055556,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.055556,0.0,0.111111,0.0,0.00,0.000000,0.000000,0.0,0.0,0.055556,0.055556,0.000000,0.0,0.0,0.0
2,Chinatown,-1.230278,0.125900,0.0,0.00,0.058824,0.0,0.058824,0.0,0.00,0.058824,0.0,0.0,0.0,0.00,0.00,0.00,0.176471,0.00,0.0,0.0,0.00,0.0,0.058824,0.00,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.058824,0.0,0.0,0.0,0.000000,0.058824,0.000000,0.0,0.0,0.00,0.000000,0.000000,0.000000,0.0,0.058824,0.117647,0.000000,0.058824,0.0,0.000000,0.0,0.117647,0.0,0.000000,0.0,0.00,0.058824,0.000000,0.0,0.0,0.000000,0.000000,0.058824,0.0,0.0,0.0
3,Clinton,1.164640,-0.174376,0.0,0.00,0.266667,0.0,0.000000,0.0,0.00,0.000000,0.0,0.0,0.0,0.00,0.00,0.00,0.000000,0.00,0.0,0.0,0.00,0.0,0.000000,0.00,0.0,0.0,0.0,0.066667,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.066667,0.000000,0.0,0.0,0.00,0.066667,0.000000,0.000000,0.0,0.133333,0.000000,0.066667,0.066667,0.0,0.066667,0.0,0.066667,0.0,0.066667,0.0,0.00,0.000000,0.066667,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0
4,East Harlem,-1.036096,-0.966235,0.0,0.00,0.000000,0.0,0.000000,0.0,0.00,0.100000,0.0,0.0,0.0,0.00,0.05,0.00,0.000000,0.05,0.0,0.0,0.05,0.0,0.000000,0.00,0.0,0.0,0.0,0.050000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.15,0.000000,0.200000,0.000000,0.0,0.000000,0.000000,0.000000,0.050000,0.0,0.000000,0.0,0.050000,0.0,0.000000,0.0,0.00,0.050000,0.050000,0.0,0.0,0.050000,0.000000,0.100000,0.0,0.0,0.0


In [85]:
# set number of clusters
kclusters = 5

manhattan_grouped_clustering = manhattan_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(manhattan_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([3, 4, 0, 1, 0, 4, 4, 0, 0, 3], dtype=int32)

In [86]:
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

In [87]:
neighborhoods_venues_sorted

,Cluster Labels,Neighborhood,price_tier,likes_count,1st Most Common Restaurant,2nd Most Common Restaurant,3rd Most Common Restaurant,4th Most Common Restaurant,5th Most Common Restaurant,6th Most Common Restaurant,7th Most Common Restaurant,8th Most Common Restaurant,9th Most Common Restaurant,10th Most Common Restaurant
0,3,Central Harlem,2.000000,37.850000,African Restaurant,Chinese Restaurant,American Restaurant,Fried Chicken Joint,French Restaurant,Seafood Restaurant,Café,Ethiopian Restaurant,Bagel Shop,Pizza Place
1,4,Chelsea,2.277778,513.611111,American Restaurant,Japanese Restaurant,Italian Restaurant,Seafood Restaurant,Chinese Restaurant,Indian Restaurant,Mexican Restaurant,Sandwich Place,Israeli Restaurant,Asian Restaurant
2,0,Chinatown,1.529412,265.823529,Chinese Restaurant,Noodle House,Sandwich Place,Pizza Place,Spanish Restaurant,Italian Restaurant,English Restaurant,New American Restaurant,Bakery,Greek Restaurant
3,1,Clinton,2.400000,204.133333,American Restaurant,New American Restaurant,Italian Restaurant,Mediterranean Restaurant,Pizza Place,Restaurant,Sandwich Place,Seafood Restaurant,Peruvian Restaurant,Steakhouse
4,0,East Harlem,1.600000,41.450000,Mexican Restaurant,Latin American Restaurant,Thai Restaurant,Bakery,Pizza Place,French Restaurant,Taco Place,Café,Steakhouse,Spanish Restaurant
5,4,East Village,1.888889,588.500000,Vietnamese Restaurant,Pizza Place,Vegetarian / Vegan Restaurant,Scandinavian Restaurant,Korean Restaurant,Ramen Restaurant,Caribbean Restaurant,Japanese Restaurant,Burger Joint,Bagel Shop
6,4,Greenwich Village,2.157895,477.578947,Italian Restaurant,Sushi Restaurant,French Restaurant,Café,Sandwich Place,Food Truck,Bagel Shop,New American Restaurant,Snack Place,Vietnamese Restaurant
7,0,Hamilton Heights,1.450000,39.800000,Café,Pizza Place,Bakery,Mexican Restaurant,Caribbean Restaurant,Gastropub,Mediterranean Restaurant,Latin American Restaurant,Burger Joint,Japanese Restaurant
8,0,Inwood,1.578947,25.947368,Mexican Restaurant,Restaurant,Café,Deli / Bodega,American Restaurant,Bakery,Spanish Restaurant,Latin American Restaurant,Diner,Pizza Place
9,3,Lenox Hill,2.052632,187.526316,Burger Joint,Thai Restaurant,Bakery,Afghan Restaurant,French Restaurant,Italian Restaurant,Japanese Restaurant,Middle Eastern Restaurant,Chinese Restaurant,Pizza Place


In [90]:
manhattan_merged = manhattan_data[manhattan_data['Neighborhood'].isin(neighborhoods_venues_sorted['Neighborhood'])]

In [91]:
manhattan_merged.shape

(24, 4)

In [92]:

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
manhattan_merged = manhattan_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

manhattan_merged.head() # check the last columns!

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,price_tier,likes_count,1st Most Common Restaurant,2nd Most Common Restaurant,3rd Most Common Restaurant,4th Most Common Restaurant,5th Most Common Restaurant,6th Most Common Restaurant,7th Most Common Restaurant,8th Most Common Restaurant,9th Most Common Restaurant,10th Most Common Restaurant
0,Manhattan,Marble Hill,40.876551,-73.910660,0,1.428571,13.857143,Sandwich Place,Deli / Bodega,Diner,Donut Shop,American Restaurant,Pizza Place,Asian Restaurant,Steakhouse,Bakery,Fast Food Restaurant
1,Manhattan,Chinatown,40.715618,-73.994279,0,1.529412,265.823529,Chinese Restaurant,Noodle House,Sandwich Place,Pizza Place,Spanish Restaurant,Italian Restaurant,English Restaurant,New American Restaurant,Bakery,Greek Restaurant
2,Manhattan,Washington Heights,40.851903,-73.936900,0,1.777778,56.166667,New American Restaurant,Deli / Bodega,Café,Bakery,Pizza Place,Breakfast Spot,Latin American Restaurant,Restaurant,Caribbean Restaurant,Italian Restaurant
3,Manhattan,Inwood,40.867684,-73.921210,0,1.578947,25.947368,Mexican Restaurant,Restaurant,Café,Deli / Bodega,American Restaurant,Bakery,Spanish Restaurant,Latin American Restaurant,Diner,Pizza Place
4,Manhattan,Hamilton Heights,40.823604,-73.949688,0,1.450000,39.800000,Café,Pizza Place,Bakery,Mexican Restaurant,Caribbean Restaurant,Gastropub,Mediterranean Restaurant,Latin American Restaurant,Burger Joint,Japanese Restaurant


# cluster map

In [93]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(manhattan_merged['Latitude'], manhattan_merged['Longitude'], manhattan_merged['Neighborhood'], manhattan_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

# Cluster 1

In [283]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 0, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,price_tier,likes_count,1st Most Common Restaurant,2nd Most Common Restaurant,3rd Most Common Restaurant,4th Most Common Restaurant,5th Most Common Restaurant,6th Most Common Restaurant,7th Most Common Restaurant,8th Most Common Restaurant,9th Most Common Restaurant,10th Most Common Restaurant
0,Marble Hill,1.428571,13.857143,Sandwich Place,American Restaurant,Deli / Bodega,Chinese Restaurant,Seafood Restaurant,Pizza Place,Vegetarian / Vegan Restaurant,Diner,Donut Shop,Steakhouse
1,Chinatown,1.529412,265.823529,Chinese Restaurant,Noodle House,Sandwich Place,Pizza Place,Spanish Restaurant,Italian Restaurant,English Restaurant,New American Restaurant,Bakery,Greek Restaurant
2,Washington Heights,1.777778,56.166667,Bakery,New American Restaurant,Deli / Bodega,Café,Pizza Place,Latin American Restaurant,Restaurant,Caribbean Restaurant,Breakfast Spot,Burger Joint
3,Inwood,1.578947,25.947368,Restaurant,Mexican Restaurant,Café,Deli / Bodega,American Restaurant,Bakery,Pizza Place,Spanish Restaurant,Diner,Latin American Restaurant
4,Hamilton Heights,1.450000,39.800000,Café,Pizza Place,Caribbean Restaurant,Bakery,Mexican Restaurant,Food Truck,Gastropub,Indian Restaurant,Italian Restaurant,Sushi Restaurant
7,East Harlem,1.600000,41.450000,Mexican Restaurant,Latin American Restaurant,Thai Restaurant,Bakery,Pizza Place,Sandwich Place,French Restaurant,Café,Taco Place,Steakhouse
11,Roosevelt Island,1.400000,8.500000,Deli / Bodega,Noodle House,Pizza Place,Greek Restaurant,Restaurant,Sandwich Place,Café,Chinese Restaurant,Japanese Restaurant,Falafel Restaurant


# Cluster 2

In [284]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 1, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,price_tier,likes_count,1st Most Common Restaurant,2nd Most Common Restaurant,3rd Most Common Restaurant,4th Most Common Restaurant,5th Most Common Restaurant,6th Most Common Restaurant,7th Most Common Restaurant,8th Most Common Restaurant,9th Most Common Restaurant,10th Most Common Restaurant
8,Upper East Side,2.550000,167.050000,Italian Restaurant,French Restaurant,Pizza Place,American Restaurant,Mexican Restaurant,Latin American Restaurant,Japanese Restaurant,Burrito Place,Salad Place,Sandwich Place
13,Lincoln Square,2.450000,282.800000,Café,American Restaurant,Italian Restaurant,Mediterranean Restaurant,French Restaurant,Food Truck,Bakery,Mexican Restaurant,Burger Joint,Seafood Restaurant
14,Clinton,2.400000,204.133333,American Restaurant,New American Restaurant,Italian Restaurant,Peruvian Restaurant,Mediterranean Restaurant,Restaurant,Sandwich Place,Seafood Restaurant,Pizza Place,Steakhouse
16,Murray Hill,2.526316,234.315789,Japanese Restaurant,Italian Restaurant,Bagel Shop,Burger Joint,Vietnamese Restaurant,Sandwich Place,Hawaiian Restaurant,Vegetarian / Vegan Restaurant,Jewish Restaurant,Mediterranean Restaurant
21,Tribeca,2.578947,332.315789,Greek Restaurant,American Restaurant,Café,Steakhouse,Bakery,Indian Restaurant,Italian Restaurant,Korean Restaurant,New American Restaurant,Salad Place


# Cluster 3

In [285]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 2, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,price_tier,likes_count,1st Most Common Restaurant,2nd Most Common Restaurant,3rd Most Common Restaurant,4th Most Common Restaurant,5th Most Common Restaurant,6th Most Common Restaurant,7th Most Common Restaurant,8th Most Common Restaurant,9th Most Common Restaurant,10th Most Common Restaurant
23,Soho,2.25,805.0,Italian Restaurant,Mediterranean Restaurant,Bakery,Spanish Restaurant,Mexican Restaurant,Pizza Place,French Restaurant,Vegetarian / Vegan Restaurant,Food Truck,Filipino Restaurant


# Cluster 4

In [286]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 3, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,price_tier,likes_count,1st Most Common Restaurant,2nd Most Common Restaurant,3rd Most Common Restaurant,4th Most Common Restaurant,5th Most Common Restaurant,6th Most Common Restaurant,7th Most Common Restaurant,8th Most Common Restaurant,9th Most Common Restaurant,10th Most Common Restaurant
5,Manhattanville,1.842105,152.421053,Italian Restaurant,Mexican Restaurant,Seafood Restaurant,Gastropub,Deli / Bodega,Cuban Restaurant,Dumpling Restaurant,Ramen Restaurant,Café,Falafel Restaurant
6,Central Harlem,2.000000,37.850000,African Restaurant,Chinese Restaurant,Seafood Restaurant,American Restaurant,Fried Chicken Joint,French Restaurant,Pizza Place,Ethiopian Restaurant,Caribbean Restaurant,Café
9,Yorkville,1.900000,87.300000,Italian Restaurant,Deli / Bodega,Vietnamese Restaurant,Bagel Shop,Mexican Restaurant,German Restaurant,Café,Sandwich Place,Burger Joint,Diner
10,Lenox Hill,2.052632,187.526316,Burger Joint,Thai Restaurant,Bakery,Afghan Restaurant,French Restaurant,Italian Restaurant,Japanese Restaurant,Middle Eastern Restaurant,Chinese Restaurant,Café
12,Upper West Side,2.000000,317.850000,Seafood Restaurant,American Restaurant,Italian Restaurant,Bakery,Southern / Soul Food Restaurant,Mediterranean Restaurant,Chinese Restaurant,Ramen Restaurant,Indian Restaurant,Vegetarian / Vegan Restaurant
15,Midtown,2.166667,143.222222,Sushi Restaurant,Steakhouse,Vietnamese Restaurant,Salad Place,Hawaiian Restaurant,French Restaurant,Food Truck,Mediterranean Restaurant,Deli / Bodega,Cuban Restaurant
20,Lower East Side,1.833333,274.722222,Café,Japanese Restaurant,Vietnamese Restaurant,French Restaurant,Filipino Restaurant,Italian Restaurant,Latin American Restaurant,Mediterranean Restaurant,Mexican Restaurant,Diner


# Cluster 5

In [287]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 4, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,price_tier,likes_count,1st Most Common Restaurant,2nd Most Common Restaurant,3rd Most Common Restaurant,4th Most Common Restaurant,5th Most Common Restaurant,6th Most Common Restaurant,7th Most Common Restaurant,8th Most Common Restaurant,9th Most Common Restaurant,10th Most Common Restaurant
17,Chelsea,2.277778,513.611111,Japanese Restaurant,Italian Restaurant,Seafood Restaurant,French Restaurant,Bakery,New American Restaurant,Chinese Restaurant,Sandwich Place,Middle Eastern Restaurant,Indian Restaurant
18,Greenwich Village,2.157895,477.578947,Italian Restaurant,Sushi Restaurant,French Restaurant,Café,New American Restaurant,Food Truck,Bagel Shop,Snack Place,Sandwich Place,English Restaurant
19,East Village,1.888889,588.500000,Vietnamese Restaurant,Vegetarian / Vegan Restaurant,Pizza Place,American Restaurant,Burger Joint,Korean Restaurant,Ramen Restaurant,Caribbean Restaurant,Japanese Restaurant,Scandinavian Restaurant
22,Little Italy,1.800000,501.250000,Café,Pizza Place,Sandwich Place,Snack Place,Italian Restaurant,Dumpling Restaurant,Mediterranean Restaurant,Noodle House,Salad Place,Seafood Restaurant
